In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import arrow
from requests.auth import AuthBase


In [3]:
def GetHistoCoinBase(tickers,startdate,enddate,timeframe):
    #date format as arg must be YYYY-MM-DD, and tickers a list of ticker(s)
    granularity = timeframe 
    for ticker in tickers:
        end_ = arrow.get(enddate).timestamp
        start_ = arrow.get(startdate).timestamp #creates a loop limit        
        date_line_limit = 200  #Coinbase limit is 200
        date_limit = end_ - (granularity*date_line_limit)
        ticker_df = []
        while date_limit >= start_:

            url = f'https://api.pro.coinbase.com/products/{ticker}/candles'.format()

            params_histo = {'granularity' : granularity,
                        'start' : arrow.get(date_limit),
                        'end' : arrow.get(end_)}

            r = requests.get(url=url,params=params_histo)
            data = r.json()
            time.sleep(0.1)

            if data is None:
                break
            elif end_ == data[-1:][0][0]:
                break
            elif date_limit == start_:
                break
            else:
                end_ = data[-1:][0][0]

            for i in data:
                ticker_df.append(i)

            date_limit = end_ - (granularity*date_line_limit)
         
        #Tidy the DataFrame
        column_names = ['time','Low','High','Open','Close','Volume']
        ticker_df.drop_duplicates(subset='time',inplace=True)
        ticker_df['Date'] = [arrow.Arrow.fromtimestamp(date).format('YYYY-MM-DD HH:mm') for date in ticker_df['time']]
        newname.drop('time',axis=1,inplace=True)
        ticker_df.sort_values(by='Date',inplace=True,ascending=True)
        ticker_df = pd.DataFrame(ticker_df,columns=columns_names)
        
        filename = [ticker.replace('/','-') if '/' in ticker else ticker][0]

        path=r'your path'
        ticker_df.to_csv(f'{path}\{filename}.csv'.format(),index=False)